# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [99]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

In [100]:
bb.get_version_string()

'4.4.0'

異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [101]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLut4Simple'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist/tb_mnist_lut4_simple'
rtl_module_name       = 'MnistLut4Simple'
output_velilog_file   = os.path.join(data_path, rtl_module_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 8
mini_batch_size       = 64
frame_modulation_size = 15

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [102]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [103]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
#           bb.DifferentiableLut([4*4*64], connection='random', N=4),
            bb.DifferentiableLut([4*64],   connection='serial', N=4),
            bb.DifferentiableLut([64],     connection='serial', N=4),

            bb.DifferentiableLut([4*64],   connection='random', N=4),
            bb.DifferentiableLut([64],     connection='serial', N=4),

            bb.DifferentiableLut([4*64],   connection='random', N=4),
            bb.DifferentiableLut([64],     connection='serial', N=4),

#           bb.DifferentiableLut([2*4*4*10], connection='random', N=4),
            bb.DifferentiableLut([4*4*10],  connection='random', N=4),
            bb.AverageLut([4*10],   connection='serial', N=4),
#           bb.DifferentiableLut([10],     connection='serial', N=4),
            bb.AverageLut([10],     connection='serial', N=4),
            bb.BinaryToReal(frame_integration_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [104]:
#bb.load_networks(data_path, net)

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(data_path, net)

  0%|          | 0/938 [00:00<?, ?it/s]

epoch[0] : loss=1.693324 accuracy=0.732000


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[1] : loss=1.798396 accuracy=0.595800


  0%|          | 0/938 [00:00<?, ?it/s]

...

epoch[5] : loss=1.800920 accuracy=0.628500


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[6] : loss=1.815398 accuracy=0.570300


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[7] : loss=1.683238 accuracy=0.776200


## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [105]:
# export verilog
bb.load_networks(data_path, net)

# 結果を出力
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_layers(f, module_name=rtl_module_name, net=net, device="", ff=False)

# Simulation用ファイルに上書きコピー
os.makedirs(rtl_sim_path, exist_ok=True)
shutil.copyfile(output_velilog_file, sim_velilog_file)

'../../verilog/mnist/tb_mnist_lut4_simple/MnistLut4Simple.v'

In [106]:
output_velilog_file

'./data/MnistDifferentiableLut4Simple/MnistLut4Simple.v'

In [107]:
# シミュレーション用データファイル作成
with open(os.path.join(rtl_sim_path, 'mnist_test.txt'), 'w') as f:
    bb.dump_verilog_readmemb_image_classification(f ,loader_test)

## モデルの内部の値を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [108]:
# 最新の保存データ読み込み
bb.load_networks(data_path, net)

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、6つの入力と接続されており、layer0 の出力ノードは 1024 個あるので、1024x6 の行列が取得できます。

In [109]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(256, 4)


array([[  0,   1,   2,   3],
       [  4,   5,   6,   7],
       [  8,   9,  10,  11],
       ...,
       [228, 229, 230, 231],
       [232, 233, 234, 235],
       [236, 237, 238, 239]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
6入力のLUTモデルなので $ 2^6 = 64 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [110]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(256, 16)


array([[ True, False,  True, ..., False,  True, False],
       [False,  True, False, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       ...,
       [ True, False,  True, ..., False, False, False],
       [False,  True,  True, ..., False,  True, False],
       [ True,  True,  True, ..., False, False, False]])

### 重み行列を覗いてみる

6入力のLUTモデルなので $ 2^6 = 64 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [111]:
W = layer0.W().numpy()
print(W.shape)
W

(256, 16)


array([[0.4922306 , 0.502768  , 0.5006771 , ..., 0.48320994, 0.50905   ,
        0.4983631 ],
       [0.4875452 , 0.50501513, 0.48546407, ..., 0.49735406, 0.5104283 ,
        0.5068313 ],
       [0.5099172 , 0.5052846 , 0.4958243 , ..., 0.50516886, 0.49422988,
        0.49619564],
       ...,
       [0.5369548 , 0.5509052 , 0.5723481 , ..., 0.3721517 , 0.49015203,
        0.45553136],
       [0.39089414, 0.55305946, 0.55199957, ..., 0.4146306 , 0.47446194,
        0.38839293],
       [0.5309876 , 0.60064775, 0.65317756, ..., 0.41775122, 0.46152145,
        0.32196406]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [112]:
# そのままだとすべて0なので、1回だけbackward実施
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

dW = layer0.dW().numpy()
print(dW.shape)
dW

(256, 16)


array([[ 1.1641532e-09,  6.6938810e-10,  6.6938810e-10, ...,
         0.0000000e+00,  0.0000000e+00, -1.8189894e-12],
       [-2.0372681e-10, -2.5465852e-11, -2.5465852e-11, ...,
        -6.8212103e-12, -6.8212103e-12, -1.3642421e-12],
       [ 9.3132257e-10,  1.1641532e-10,  1.1641532e-10, ...,
         2.1827873e-11,  2.1827873e-11,  7.2759576e-12],
       ...,
       [ 3.9865403e-04,  1.3674135e-04,  5.2335439e-05, ...,
         2.9016963e-05, -5.7820731e-04, -8.8611232e-05],
       [-3.2140375e-03, -4.2136726e-03, -1.2364655e-03, ...,
         7.4813637e-05,  2.6108748e-03,  2.6727887e-03],
       [-4.0364992e-03,  4.0049516e-03,  1.2718013e-04, ...,
        -7.5191667e-04, -1.0017462e-03,  3.4563802e-04]], dtype=float32)